In [11]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, datasets, linear_model, model_selection
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
from cmath import sqrt
import pickle

In [12]:
dataframe=pd.read_csv('../Dataset.csv')

Видалимо всі рядки з порожніми даними, та закодуємо стовпчик з фічею 'platform'.

In [13]:
dataframe.drop(dataframe.index[dataframe.isna().any(axis=1)].tolist(),axis=0,inplace=True)
income_dataframe=dataframe.filter(['target_sub_ltv_day30','target_iap_ltv_day30','target_ad_ltv_day30','target_full_ltv_day30'],axis=1)
dataframe.drop(columns=['target_sub_ltv_day30','target_iap_ltv_day30','target_ad_ltv_day30','target_full_ltv_day30'],axis=1,inplace=True)

dataframe.drop(columns=['media_source','install_date','country_code'],axis=1,inplace=True)
dummies_platform=pd.get_dummies(dataframe['platform'],drop_first=True)
dataframe.drop(columns=['platform'],axis=1,inplace=True)
dataframe=pd.concat([dataframe,dummies_platform],axis='columns')
print(dataframe)

         total_sessions_day0  total_sessions_day1  total_sessions_day3  \
0                        1.0                  1.0                  1.0   
1                        1.0                  1.0                  1.0   
2                        1.0                  2.0                  2.0   
3                        1.0                  1.0                  1.0   
4                        1.0                  1.0                  1.0   
...                      ...                  ...                  ...   
1467827                 11.0                 11.0                 11.0   
1467828                  2.0                  3.0                  6.0   
1467829                  3.0                  5.0                 11.0   
1467830                  5.0                  5.0                  5.0   
1467831                  6.0                  7.0                  7.0   

         total_sessions_day7  chapters_finished_day0  chapters_finished_day1  \
0                        1.0   

In [14]:
new_dataframe=dataframe.copy()

Натренуємо модель на всіх фічах для того, щоб на основі абсолютних значень коефіцієнтів впорядкувати фічі за "важливістю".

In [15]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(new_dataframe,income_dataframe['target_full_ltv_day30'], test_size=0.2)
model=linear_model.LinearRegression(fit_intercept=1)
model.fit(X_train, y_train)
coef=[]
for i in range(len(model.coef_)):
    coef.append((new_dataframe.columns.values[i], abs(model.coef_[i])))
coef=sorted(coef, key=lambda tup: tup[1])
coef.reverse()
for i in range(len(model.coef_)):
    print(coef[i])

('ad_ltv_day3', 4.565020491918325)
('ad_ltv_day1', 2.2671459408930956)
('app_iap_ltv_day3', 1.552644923663874)
('app_sub_ltv_day3', 1.3411146835409573)
('app_sub_ltv_day0', 1.2016637530328833)
('ad_ltv_day0', 1.0486103683311014)
('retained_day7', 0.3353853496595691)
('app_iap_ltv_day1', 0.33184202567243937)
('app_sub_ltv_day1', 0.23951449142009323)
('retained_day3', 0.20359274703188124)
('chapters_closed_day3', 0.17369295544955443)
('chapters_closed_day7', 0.17244488044279763)
('chapters_finished_day3', 0.11258189390362468)
('app_iap_ltv_day0', 0.11256527345463507)
('retained_day1', 0.08473053182496304)
('chapters_finished_day7', 0.0845457826910029)
('chapters_closed_session9', 0.07235202527158385)
('tickets_spent_day3', 0.0635731602327001)
('tickets_spent_day1', 0.05625630274881627)
('chapters_opened_session9', 0.054840480832172245)
('chapters_finished_session9', 0.0428558546290414)
('chapters_opened_day3', 0.04150153060955853)
('chapters_opened_day7', 0.04091992396646649)
('tickets_s

Будемо поступово(за "важливістю") додавати фічі в модель і у випадку покращення метрики MAE залишати їх в моделі.

In [16]:
features=[]
min=1000000
best_features=[]
for x in coef:
    name=x[1]
    features.append(x[0])
    training_dataframe=new_dataframe.filter(features,axis='columns')
    X_train, X_test, y_train, y_test = model_selection.train_test_split(training_dataframe,income_dataframe['target_full_ltv_day30'], test_size=0.2)
    model=linear_model.LinearRegression(fit_intercept=1)    
    model.fit(X_train, y_train)
    predictions=model.predict(X_test)
    a=mean_absolute_error(predictions,y_test)
    if a<min:
        best_features=features.copy()
        min=a
        print(x[0],a)
    else:
        features.remove(x[0])

    


ad_ltv_day3 0.3755406428680663
app_iap_ltv_day3 0.13722054405248418
ad_ltv_day0 0.1321743193897526
app_iap_ltv_day1 0.12832310024225144
retained_day3 0.1274043493298607
chapters_opened_day3 0.12538111108087094
diamonds_received_day1 0.12486298922629106


In [17]:
print(best_features,min)

['ad_ltv_day3', 'app_iap_ltv_day3', 'ad_ltv_day0', 'app_iap_ltv_day1', 'retained_day3', 'chapters_opened_day3', 'diamonds_received_day1'] 0.12486298922629106


In [18]:
training_dataframe=new_dataframe.filter(best_features,axis='columns')
X_train, X_test, y_train, y_test = model_selection.train_test_split(training_dataframe,income_dataframe['target_full_ltv_day30'], test_size=0.2)
model=linear_model.LinearRegression(fit_intercept=1)    
model.fit(X_train, y_train)
predictions=model.predict(X_test)
print(mean_absolute_error(predictions,y_test))
print(sqrt(mean_squared_error(predictions,y_test)))
print(mean_absolute_percentage_error(predictions,y_test))

0.13158900668720816
(1.361309109456833+0j)
64.95497390310743


Водночас бачимо, що модель з фічами 'ad_ltv_day3', 'ad_ltv_day1', 'app_iap_ltv_day3' має майже таку ж метрику MAE. Враховуючи той факт, що фічі 'ad_ltv_day3' та 'ad_ltv_day1' корелюють (як видно з EDA) то можна спробувати викинути одну з них. Викинемо 'ad_ltv_day1', адже вона має меншу "важливість".

In [19]:
training_dataframe=new_dataframe.filter(['ad_ltv_day3','app_iap_ltv_day3'],axis='columns')
print(training_dataframe)
X_train, X_test, y_train, y_test = model_selection.train_test_split(training_dataframe,income_dataframe['target_full_ltv_day30'], test_size=0.2)
model=linear_model.LinearRegression(fit_intercept=1)    
model.fit(X_train, y_train)
predictions=model.predict(X_test)
print(mean_absolute_error(predictions,y_test))
print(sqrt(mean_squared_error(predictions,y_test)))
print(mean_absolute_percentage_error(predictions,y_test))

         ad_ltv_day3  app_iap_ltv_day3
0           0.000000          0.000000
1           0.000000          0.000000
2           0.000000          0.000000
3           0.000000          0.000000
4           0.000000          0.000000
...              ...               ...
1467827     0.037158          4.174198
1467828     0.091267          9.404430
1467829     0.000000         13.928170
1467830     0.177550          7.402224
1467831     0.114909          3.949385

[1466458 rows x 2 columns]
0.13895927096860775
(1.2025906262861348+0j)
2.7873475818351787


Бачимо що метрика MAE не набагато більша за модель з best_features тому виберемо простішу модель.

In [22]:
print(model.coef_)
print(model.intercept_)

[3.28282544 1.48532095]
-0.00828463425962267
['ad_ltv_day3' 'app_iap_ltv_day3']


In [ ]:
saved_model=pickle.dumps(model)
f=open('model','wb')
f.write(saved_model)
f.close()